# Recipes

This page references recipes for common operations for the PyHMMER API, similarly to the `itertools` [recipes](https://docs.python.org/3/library/itertools.html#itertools-recipes) from the Python documentation.

## Loading multiple HMMs

An adapter for loading several `HMMFile` objects into a single object to pass to .

<div class="alert alert-info">
    
Credits
    
The original implementation proposed by [Zachary Kurtz](https://github.com/zdk123) in [#24](https://github.com/althonos/pyhmmer/issues/24), which was failing when too many file-descriptors where open on some OS ([#48](https://github.com/althonos/pyhmmer/issues/24)), was updated to keep at most a single file open at a time.
    
</div>

In [ ]:
import contextlib
import itertools
import os
import typing

from pyhmmer.plan7 import HMMFile, HMM

class HMMFiles(typing.Iterable[HMM]):
    def __init__(self, *files: typing.Union[str, bytes, os.PathLike]):
        self.files = files
    
    def __iter__(self):
        for file in self.files:
            with HMMFile(file) as hmm_file:
                yield from hmm_file

To use it with `hmmsearch`, simply create a `HMMFiles` object with the paths for the different HMM files to concatenate. They will be read in the order given as argument.

In [ ]:
import pyhmmer
from pyhmmer.easel import SequenceFile

with SequenceFile("data/seqs/938293.PRJEB85.HG003687.faa", digital=True) as sequences:
    targets = sequences.read_block()

hmm_files = HMMFiles("data/hmms/txt/PKSI-AT.hmm", "data/hmms/txt/LuxC.hmm")
all_hits = list(pyhmmer.hmmsearch(hmm_files, targets))
    
print("HMMs searched:", len(all_hits))
print("Hits found:   ", sum(len(hits) for hits in all_hits))

If your filenames are stored in a list, just use the [*splat* operator](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists) to unpack it:

In [ ]:
filenames = ["data/hmms/txt/PKSI-AT.hmm", "data/hmms/txt/LuxC.hmm"]
hmm_files = HMMFiles(*filenames)